In [1]:
import json
import time
import re
import urllib
from datetime import date
import polars as pl
import numpy as np
import textdistance as td
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tobias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
schema={
    'id': pl.Utf8,
    'abstract': pl.Utf8,
    'beratungsstand': pl.Utf8,
    'vorgangstyp': pl.Utf8,
    'gesta': pl.Utf8,
    'archiv': pl.Utf8,
    'sachgebiet': pl.List(pl.Utf8),
    'typ': pl.Utf8,
    'wahlperiode': pl.UInt64,
    'zustimmungsbeduerftigkeit': pl.List(pl.Utf8),
    'initiative': pl.List(pl.Utf8),
    'deskriptor': pl.List(pl.Struct([
        pl.Field('fundstelle', pl.Boolean),
        pl.Field('name', pl.Utf8),
        pl.Field('typ', pl.Utf8),
    ])),
    'aktualisiert': pl.Utf8,
    'titel': pl.Utf8,
    'datum': pl.Utf8,
    'verkuendung': pl.List(pl.Struct([
        pl.Field('verkuendungsdatum', pl.Utf8)
    ]))
}
df_vorgang = pl.DataFrame(json.load(open('vorgang.json', encoding='utf-8')), schema=schema)
df_gesetze = df_vorgang.filter(pl.col('vorgangstyp') == 'Gesetzgebung')
pl.Config(fmt_str_lengths=100)
df_gesetze['beratungsstand'].unique()

beratungsstand
str
"""Abgelehnt"""
"""Verkündet"""
"""Den Ausschüssen zugewiesen"""
"""Einbringung beschlossen"""
"""Noch nicht beraten"""
…
"""Überwiesen"""
"""Bundesrat hat zugestimmt"""
"""Beschlussempfehlung liegt vor"""


In [3]:
df_vorgangspos = pl.read_json('vorgangsposition.json', infer_schema_length=5000)
df_vorgangspos = df_vorgangspos.with_columns(
   pl.col("datum").str.to_date(format="%Y-%m-%d")
)
df_vorgangspos_gesetzgebung = df_vorgangspos.filter((pl.col('vorgangstyp') == 'Gesetzgebung') & (pl.col('typ') == 'Vorgangsposition'))
df_vorgang_duration = df_vorgangspos_gesetzgebung.group_by(['vorgang_id']).agg([
    pl.col('datum').min().alias('vorgang_start'), 
])

In [4]:
df_gesetze.head()

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]]
"""322883""","""Feststellung des Bundeshaushaltsplans 2025 in Einnahmen und Ausgaben auf 503,006 Mrd Euro, Feststell…","""Beschlussempfehlung liegt vor""","""Gesetzgebung""","""D005""",null,"[""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",21,"[""Nein, laut Gesetzentwurf (Drs 277/25)""]","[""Bundesregierung""]","[{false,""Aufbauhilfefonds 2021"",""Institutionen""}, {true,""Bundeshaushalt 2025"",""Sachbegriffe""}, … {false,""Steueraufkommen"",""Sachbegriffe""}]","""2025-08-01T15:05:36+02:00""","""Gesetz über die Feststellung des Bundeshaushaltsplans für das Haushaltsjahr 2025 (Haushaltsgesetz 20…","""2025-08-01""",null
"""323276""","""Aufteilung des neu eingeführten strukturellen Verschuldungsspielraums (Strukturkomponente) für die L…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""D008""",null,"[""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",21,"[""Ja, laut Gesetzentwurf (Drs 315/25)""]","[""Bundesregierung""]","[{true,""Gesetz zur Ausführung von Artikel 109 Absatz 3 Satz 6 und Satz 7 des Grundgesetzes und zur Änderung anderer Gesetze"",""Rechtsmaterialien""}, {false,""Haushaltsgrundsätzegesetz"",""Rechtsmaterialien""}, … {false,""Öffentliche Finanzkontrolle"",""Sachbegriffe""}]","""2025-07-30T17:39:06+02:00""","""Gesetz zur Ausführung von Artikel 109 Absatz 3 Satz 6 und Satz 7 des Grundgesetzes und zur Änderung …","""2025-07-30""",null
"""323274""","""Umsetzung des für Länder und Kommunen gem. Artikel 143h des Grundgesetzes vorgesehenen Anteils an de…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""D007""",null,"[""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",21,"[""Ja, laut Gesetzentwurf (Drs 314/25)""]","[""Bundesregierung""]","[{false,""Bericht"",""Sachbegriffe""}, {false,""Bildungseinrichtung"",""Sachbegriffe""}, … {true,""Öffentliche Investition"",""Sachbegriffe""}]","""2025-07-30T17:22:02+02:00""","""Gesetz zur Finanzierung von Infrastrukturinvestitionen von Ländern und Kommunen (Länder-und-Kommunal…","""2025-07-30""",null
"""322623""","""Einführung einer befristeten Experimentierklausel zur Zulässigkeit von Wohnbauvorhaben auch ohne Auf…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""P001""",null,"[""Raumordnung, Bau- und Wohnungswesen""]","""Vorgang""",21,"[""Nein, laut Gesetzentwurf (Drs 256/25)""]","[""Bundesregierung""]","[{false,""Baugenehmigung"",""Sachbegriffe""}, {false,""Baugesetzbuch"",""Rechtsmaterialien""}, … {true,""Wohnungsbau"",""Sachbegriffe""}]","""2025-07-30T17:15:01+02:00""","""Gesetz zur Beschleunigung des Wohnungsbaus und zur Wohnraumsicherung""","""2025-07-30""",null
"""321754""","""Zur Erfüllung des Rechtsanspruchs auf Ganztagsbetreuung für Kinder im Grundschulalter während der Sc…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""I001""",null,"[""Bildung und Erziehung"", ""Gesellschaftspolitik, soziale Gruppen""]","""Vorgang""",21,"[""Ja, laut Gesetzesantrag (Drs 208/25)""]","[""Niedersachsen"", ""Bayern"", … ""Rheinland-Pfalz""]","[{false,""Bürokratie"",""Sachbegriffe""}, {false,""Ganztagsförderungsgesetz"",""Rechtsmaterialien""}, … {false,""Statistik"",""Sachbegriffe""}]","""2025-07-31T14:54:52+02:00""","""... Gesetz zur Änderung des Achten Buches Sozialgesetzbuch - Kinder- und Jugendhilfe - Rechtsanspru…","""2025-07-30""",null


In [5]:
df_gesetze = df_gesetze.join(df_vorgang_duration.select(['vorgang_id', 'vorgang_start']), left_on='id', right_on='vorgang_id', how='left')

In [6]:
#df_vorgang_docs = df_vorgangspos.filter(pl.col('dokumentart') == 'Drucksache').group_by('vorgang_id').agg(pl.col('fundstelle').struct.field("dokumentnummer").explode())
#df_gesetze = df_gesetze.join(df_vorgang_docs.select(['vorgang_id', 'dokumentnummer']), left_on='id', right_on='vorgang_id', how='left')

In [7]:
today = date.today()
df_gesetze = df_gesetze.with_columns(
    pl.when(pl.col('beratungsstand') == 'Verkündet').then(
        pl.col('verkuendung').list.eval(pl.col('').struct.field('verkuendungsdatum').str.to_date(format="%Y-%m-%d")).list.min()
    ).when(pl.col('beratungsstand') == 'Abgelehnt').then(
        pl.col('datum').str.to_date(format="%Y-%m-%d")
    ).otherwise(today).alias('vorgang_end')
)
df_gesetze = df_gesetze.with_columns((pl.col('vorgang_end') - pl.col('vorgang_start')).dt.total_days().alias('vorgangsdauer'))

In [8]:
reg_frag = ['Fraktion der CDU/CSU', 'Fraktion der SPD']

def fix_initiative(x):
    return pl.when(x.list.set_intersection(reg_frag).len == len(reg_frag)).then(
        x.list.set_difference(reg_frag)
    ).otherwise(x)

df_gesetze = df_gesetze.with_columns(pl.when(pl.col('initiative').list.set_intersection(reg_frag).list.len() == len(reg_frag)).then(
        pl.col('initiative').list.set_difference(reg_frag).list.concat(pl.Series(['Bundesregierung']))
    ).otherwise(pl.col('initiative')).alias('initiative'))

In [9]:
df_gesetze = df_gesetze.with_columns(pl.col('deskriptor').list.eval(pl.element().struct.field('name')).alias('keywords'))

In [10]:
df_gesetze.select(pl.col("*").exclude("abstract", "vorgangstyp", "gesta", "archiv", "typ", "deskriptor")).write_json('gesetze.json')

In [11]:
df_gesetze_public = df_gesetze.filter(pl.col('beratungsstand') == 'Verkündet')
bins = [7, 14, 30, 60, 120, 240, 365]
duration_bin = np.digitize(df_gesetze_public.select('vorgangsdauer').to_series(), bins, right=True)
df_gesetze_public.select('vorgangsdauer').to_series().hist(bins)

breakpoint,category,count
f64,cat,u32
14.0,"""[7.0, 14.0]""",0
30.0,"""(14.0, 30.0]""",0
60.0,"""(30.0, 60.0]""",9
120.0,"""(60.0, 120.0]""",2
240.0,"""(120.0, 240.0]""",0
365.0,"""(240.0, 365.0]""",0


In [12]:
df_gesetze_public_bins = df_gesetze_public.with_columns(pl.from_numpy(duration_bin, schema=['duration_bin']))
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').min())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,keywords,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]],date,date,i64,list[str],i64
"""322261""","""Zur Verlangsamung des Mietenanstiegs Verlängerung der Regelungen zur Beschränkung der Miethöhe bei M…","""Verkündet""","""Gesetzgebung""","""C008""",null,"[""Raumordnung, Bau- und Wohnungswesen"", ""Recht""]","""Vorgang""",21,"[""Nein, laut Gesetzentwurf (Drs 21/322)"", ""Nein, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Bundesverfassungsgericht"",""Institutionen""}, {false,""Bürgerliches Gesetzbuch"",""Rechtsmaterialien""}, … {false,""Verfassungsgerichtsbarkeit"",""Sachbegriffe""}]","""2025-07-22T15:24:24+02:00""","""Gesetz zur Änderung der Regelungen über die zulässige Miethöhe bei Mietbeginn und zur Änderung ander…","""2025-07-11""","[{""2025-07-22""}]",2025-06-03,2025-07-22,49,"[""Bundesverfassungsgericht"", ""Bürgerliches Gesetzbuch"", … ""Verfassungsgerichtsbarkeit""]",3
"""322260""","""Aufnahme der Begrenzung der Zuwanderung als ausdrücklich genannte Zielbestimmung des Aufenthaltsrech…","""Verkündet""","""Gesetzgebung""","""B007""",null,"[""Migration und Aufenthaltsrecht""]","""Vorgang""",21,"[""Nein, laut Gesetzentwurf (Drs 21/321)"", ""Nein, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Aufenthaltsgesetz"",""Rechtsmaterialien""}, {false,""Aufenthaltsrecht"",""Sachbegriffe""}, … {true,""Gesetz zur Aussetzung des Familiennachzugs zu subsidiär Schutzberechtigten"",""Rechtsmaterialien""}]","""2025-07-24T09:06:12+02:00""","""Gesetz zur Aussetzung des Familiennachzugs zu subsidiär Schutzberechtigten""","""2025-07-11""","[{""2025-07-23""}]",2025-06-03,2025-07-23,50,"[""Aufenthaltsgesetz"", ""Aufenthaltsrecht"", … ""Gesetz zur Aussetzung des Familiennachzugs zu subsidiär Schutzberechtigten""]",3
"""322259""","""Umsetzung folgender Vorgaben des Bundesverfassungsgerichts: Zulässigkeit heimlicher Datenerhebung be…","""Verkündet""","""Gesetzgebung""","""B006""",null,"[""Innere Sicherheit""]","""Vorgang""",21,"[""Ja, laut Gesetzentwurf (Drs 21/325)"", ""Ja, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Bundeskriminalamtgesetz"",""Rechtsmaterialien""}, {false,""Bundesverfassungsgericht"",""Institutionen""}, … {true,""Terrorismusbekämpfung"",""Sachbegriffe""}]","""2025-07-24T09:04:53+02:00""","""Gesetz zur Anpassung der Befugnis zur Datenerhebung bei Kontaktpersonen im Bundeskriminalamtgesetz""","""2025-07-11""","[{""2025-07-23""}]",2025-06-03,2025-07-23,50,"[""Bundeskriminalamtgesetz"", ""Bundesverfassungsgericht"", … ""Terrorismusbekämpfung""]",3
"""322258""",null,"""Verkündet""","""Gesetzgebung""","""B005""",null,"[""Innere Sicherheit""]","""Vorgang""",21,"[""Nein, laut Gesetzentwurf (Drs 21/324)"", ""Nein, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Bundeskriminalamt"",""Institutionen""}, {false,""Bundeskriminalamtgesetz"",""Rechtsmaterialien""}, … {false,""Waffengesetz"",""Rechtsmaterialien""}]","""2025-07-24T09:03:29+02:00""","""Gesetz zur Anpassung von Regelungen über den polizeilichen Informationsverbund im Bundeskriminalamtg…","""2025-07-11""","[{""2025-07-23""}]",2025-06-03,2025-07-23,50,"[""Bundeskriminalamt"", ""Bundeskriminalamtgesetz"", … ""Waffengesetz""]",3
"""322257""","""Verlängerung der Fristen für die Umsetzung der verpflichtenden Kennzeichnung um sieben Monate auf de…","""Verkündet""","""Gesetzgebung""","""F002""",null,"[""Landwirtschaft und Ernährung""]","""Vorgang""",21,"[""Nein, laut Gesetzentwurf (Drs 21/327)"", ""Nein, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Gesetzgebung"",""Sachbegriffe""}, {true,""Lebensmittelkennzeichnung"",""Sachbegriffe""}, … {true,""Viehwirtschaft"",""Sachbegriffe""}]","""2025-07-22T15:36:51+02:00""","""Erstes Gesetz 

In [13]:
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').max())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,keywords,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]],date,date,i64,list[str],i64
"""321950""","""Anpassung nationaler Bestimmungen an EU-Recht und Beseitigung von in der Anwendung erkannten Unschär…","""Verkündet""","""Gesetzgebung""","""O001""",null,"[""Kultur""]","""Vorgang""",21,"[""Ja, laut Gesetzentwurf (Drs 21/219)"", ""Ja, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Artenschutz"",""Sachbegriffe""}, {false,""Ausfuhr"",""Sachbegriffe""}, … {false,""Zuständigkeit"",""Sachbegriffe""}]","""2025-07-22T15:46:51+02:00""","""Erstes Gesetz zur Änderung des Kulturgutschutzgesetzes (1. KGSGÄndG)""","""2025-07-11""","[{""2025-07-22""}]",2025-05-20,2025-07-22,63,"[""Artenschutz"", ""Ausfuhr"", … ""Zuständigkeit""]",4
"""321946""","""Fristverlängerung des Förder- und Abrechnungszeitraums des Investitionsprogramms zum beschleunigten …","""Verkündet""","""Gesetzgebung""","""I002""",null,"[""Bildung und Erziehung"", ""Gesellschaftspolitik, soziale Gruppen"", ""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",21,"[""Ja, laut Gesetzentwurf (Drs 21/216)"", ""Ja, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Bildungsfinanzierung"",""Sachbegriffe""}, {false,""Bundesministerium für Bildung, Familie, Senioren, Frauen und Jugend"",""Institutionen""}, … {false,""Öffentliche Investition"",""Sachbegriffe""}]","""2025-07-24T09:07:49+02:00""","""Gesetz zur Verlängerung der Fristen im Investitionsprogramm Ganztagsausbau""","""2025-07-11""","[{""2025-07-23""}]",2025-05-20,2025-07-23,64,"[""Bildungsfinanzierung"", ""Bundesministerium für Bildung, Familie, Senioren, Frauen und Jugend"", … ""Öffentliche Investition""]",4


In [14]:
def map_gesetze(tokens):
    overlap = df_gesetze.with_columns(pl.col('titel.stem').map_elements(lambda x: td.lcsstr.normalized_similarity(tokens, x), return_dtype=pl.Float32).alias('similarity')).select(['titel', 'titel.stem', 'id', 'similarity'])
    overlap_ = overlap.sort('similarity', descending=True)
    overlap = overlap_.filter(pl.col('similarity') > 0.5)
    if overlap.shape[0] == 0:
        print(f'Warning: Count not find any Gesetze with similarity > 0.5: {tokens}')
        print(overlap_.head())
        return None
    if overlap.shape[0] > 1:
        if overlap.row(1, named=True)['similarity'] * 2 > overlap.row(0, named=True)['similarity'] and overlap.row(0, named=True)['similarity'] != 1:
            print(f'Error: could not map gesetz: {tokens}')
            print(overlap.head())
            return None
    #print(tokens)
    #print(overlap.head())
    return overlap.row(0, named=True)

bad_words = [
    'beratung',
    'gebrach',
    'entwurf',
    'eine'
]
stopwords_german = stopwords.words('german')
def rm_stop_words(txt):
    words = txt.split()
    words = list(filter(lambda x: x not in stopwords_german, words))
    return ' '.join(words)
    
def pre_process(txt):
    txt = txt.split('eingebrach', 1)[1]
    txt = ' '.join(txt.split())
    txt = re.sub(r'[a-z]\)\(', '', txt)
    txt = re.sub(r'des von (der|den) .*(fraktio|bundesregierung)', '', txt)
    txt = re.sub(r'^–', '', txt)
    for word in bad_words:
        txt = txt.replace(word, '', 1)
    return rm_stop_words(txt)

stemmer = Cistem()
def stem(txt):
    words = []
    for word in txt.split():
        words.append(stemmer.segment(word)[0])
    return ' '.join(words)

def parse_top_details(details):
    top_l = []
    for top in str(details).split('<br/><br/>'):
        top = BeautifulSoup(top, 'lxml')
        vorgang_id = None
        for a in top.find_all('a'):
            drucksache = df_vorgangspos.filter((pl.col('dokumentart') == 'Drucksache') & (pl.col('fundstelle').struct.field("drucksachetyp") == 'Gesetzentwurf') & (pl.col('fundstelle').struct.field("pdf_url") == a['href']))
            if drucksache.shape[0] == 0:
                continue
            if drucksache.shape[0] == 1:
                vorgang_id_new = drucksache.row(0, named=True)['vorgang_id']
                if vorgang_id and vorgang_id != vorgang_id_new:
                    raise RuntimeError('vorgang id missmatch')
                vorgang_id = vorgang_id_new
            if drucksache.shape[0] > 1:
                print(drucksache)
                raise RuntimeError('Found multiple vorgang ids for document')
        if vorgang_id:
            gesetz = df_gesetze.filter(pl.col('id') == vorgang_id).row(0, named=True)
            print('* ' + gesetz['titel'])
            top_l.append(gesetz)
        else:
            txt_ = top.get_text(' ')
            for txt in txt_.split('Drucksache'):
                if 'gesetz' in txt.lower() and 'entwurf' in txt.lower():
                    txt = stem(txt)
                    txt = pre_process(txt)
                    gesetz = map_gesetze(txt)
                    if gesetz is not None:
                        print('* ' + gesetz['titel'])
                        top_l.append(gesetz)
    return top_l

def map_top_gesetze(soup):
    tagesordnung = {}
    for conf in soup.find_all('div', {'class': 'bt-conference-title'}):
        sitzung = conf.text.strip()
        print(sitzung)
        top_l = []
        for thema in conf.parent.parent.find_all('td', {'data-th': 'Thema'}):
            if thema.p:
                top_l += parse_top_details(thema.p)
        print()
        tagesordnung[sitzung] = top_l
    return tagesordnung

def get_tagesordnung_gesetze_html(kw):
    res = requests.get('https://www.bundestag.de/apps/plenar/plenar/conferenceweekDetail.form', {'year': 2025, 'week': kw, 'limit': 10})
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'lxml')
    return map_top_gesetze(soup)

def get_tagesordnung_gesetze_json(kw=None, year=None):
    args = None
    if kw and year:
        args = {'year': year, 'week': kw}
    res = requests.get('https://www.bundestag.de/apps/plenar/plenar/conferenceWeekJSON', args)
    time.sleep(0.3)
    return res.json()

def get_tagesordnung_gesetze():
    tagesordnung = {}
    current_week = get_tagesordnung_gesetze_json()
    conferences = current_week["conferences"]
    if "previous" in current_week:
        prev_week = get_tagesordnung_gesetze_json(current_week["previous"]["week"], current_week["previous"]["year"])
        conferences += prev_week["conferences"]
    if "next" in current_week:
        next_week = get_tagesordnung_gesetze_json(current_week["next"]["week"], current_week["next"]["year"])
        conferences += next_week["conferences"]
    conferences = sorted(conferences, key=lambda x: x["conferenceNumber"])
    for day in conferences:
        sitzung = f'{day["conferenceDate"]["date"]} ({day["conferenceNumber"]}. Sitzung)'
        print(sitzung)
        top_l = []
        for top in day["rows"]:
            topic = top["topic"]
            title = topic["title"]
            detail = None
            link = None
            if "detail" in topic:
                detail = topic["detail"]
            if "link" in topic:
                link = topic["link"]
            top_l += parse_top_details(detail)
        tagesordnung[sitzung] = top_l
        print()
    return tagesordnung
    

In [15]:
df_gesetze = df_gesetze.with_columns(pl.col('titel').map_elements(stem, return_dtype=pl.self_dtype()).map_elements(rm_stop_words, return_dtype=pl.self_dtype()).alias('titel.stem'))

In [16]:
#df_vorgangspos.filter((pl.col('dokumentart') == 'Drucksache')).select(pl.col('fundstelle').struct.field("pdf_url"))
#df_vorgangspos.filter(pl.col('fundstelle').struct.field("pdf_url").eq('https://dserver.bundestag.de/btd/20/106/2010664.pdf'))

In [17]:
top = get_tagesordnung_gesetze()

8. Juli 2025 (16. Sitzung)
* Gesetz über die Feststellung des Bundeshaushaltsplans für das Haushaltsjahr 2025 (Haushaltsgesetz 2025 - HG 2025)
* Haushaltsbegleitgesetz 2025

9. Juli 2025 (17. Sitzung)

10. Juli 2025 (18. Sitzung)
* Gesetz zur Verbesserung der Rahmenbedingungen für die Erprobung von Innovationen in Reallaboren und zur Förderung des regulatorischen Lernens
* Gesetz zur Änderung des Bundes-Immissionsschutzgesetzes und weiterer Vorschriften
* Gesetz zur Bestimmung sicherer Herkunftsstaaten durch Rechtsverordnung und Abschaffung des anwaltlichen Vertreters bei Abschiebungshaft und Ausreisegewahrsam
* Gesetz zur Beschleunigung des Wohnungsbaus und zur Wohnraumsicherung
* Gesetz zur Umsetzung von Vorgaben der Richtlinie (EU) 2023/2413 für Zulassungsverfahren nach dem Bundes-Immissionsschutzgesetz und dem Wasserhaushaltsgesetz sowie für Planverfahren nach dem Baugesetzbuch und dem Raumordnungsgesetz, zur Änderung des Bundeswasserstraßengesetzes und zur Änderung des Windenergie

In [18]:
json.dump({day: [gesetz['id'] for gesetz in value] for day, value in top.items()}, open('tagesordnung.json', 'wt'))

In [19]:
pre_process(stem('Erste Beratung des von der CDU/CSU-Fraktion eingebrachten Gesetzentwurfs zur rechtssicheren Einführung '))

'gesetz rechtssich einführung'

In [20]:
pre_process(stem(' – Zweite und dritte Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes zur Durchführung'))

'gesetz durchführung'

In [21]:
pre_process(stem('a) Zweite und dritte Beratung des von der Bundesregierung eingebrachten Entwurfs eines Zweiten Gesetzes zur Änderung des Umweltstatistikgesetzes  '))

'zweit gesetz änderung umweltstatistikgesetz'